In [9]:
using CSV, DataFrames
using FileIO, Dates
using Plots
using StatsBase

In [2]:
include("./src/cubes.jl")
include("./src/network.jl")

volume_tetrahedrons (generic function with 1 method)

In [3]:
path = "./data/"
name = "vrancea"
filepath = path * name * ".csv"

df = CSV.read(filepath, DataFrame);

df, df_cubes = region_cube_split(df,side=5,energyRelease=true);

MG = create_network(vrancea, vrancea_cubes; edgeWeight=false);
connectivity = degree(MG);

"./data/romania.csv"

In [10]:
bstep = 2 * iqr(fit_data) * length(fit_data) ^ (-1/3)
b = minimum(fit_data) : bstep : maximum(fit_data)
h = StatsBase.fit(Histogram{Float64}, fit_data, b)

Histogram{Float64, 1, Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}}
edges:
  1.0:1.333590634253139:81.01543805518834
weights: [762.0, 6.0, 248.0, 139.0, 7.0, 86.0, 71.0, 10.0, 59.0, 32.0  …  0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
closed: left
isdensity: false

In [12]:
plot(h ; label="",xscale=:log10, xlim=extrema(b))
savefig("./results/vrancea_bins_iqr.png")

"d:\\projects\\seismicNetworks-extras\\seismic-networks-julia-tests\\results\\vrancea_bins_iqr.png"

In [24]:
h = StatsBase.fit(Histogram{Float64}, fit_data, nbins=Int(ceil(sqrt(length(fit_data)))))

Histogram{Float64, 1, Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}}
edges:
  0.0:2.0:84.0
weights: [3.0, 765.0, 248.0, 146.0, 86.0, 81.0, 59.0, 38.0, 43.0, 41.0  …  3.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]
closed: left
isdensity: false

In [25]:
plot(h ; label="",xscale=:log10, xlim=extrema(b))
savefig("./results/vrancea_bins_sqrt.png")

"d:\\projects\\seismicNetworks-extras\\seismic-networks-julia-tests\\results\\vrancea_bins_sqrt.png"

In [26]:
using DataStructures

In [27]:
c = DataStructures.counter(fit_data)

Accumulator{Int64, Int64} with 68 entries:
  5  => 4
  56 => 1
  35 => 4
  55 => 1
  60 => 1
  30 => 4
  32 => 3
  6  => 139
  67 => 1
  45 => 5
  73 => 1
  64 => 2
  4  => 244
  13 => 13
  54 => 2
  63 => 1
  62 => 1
  58 => 6
  52 => 2
  ⋮  => ⋮

In [29]:
bar(collect(keys(c)), collect(values(c)),yscale=:log10)
savefig("./results/vrancea_actual_degrees.png")

"d:\\projects\\seismicNetworks-extras\\seismic-networks-julia-tests\\results\\vrancea_actual_degrees.png"

In [30]:
using PyCall

In [31]:
powerlaw = pyimport("powerlaw")

PyObject <module 'powerlaw' from 'C:\\Users\\gabipana\\.julia\\conda\\3\\lib\\site-packages\\powerlaw.py'>

In [32]:
py"""
import powerlaw as powerlaw
def fit_data_powerlaw(data):
    results = powerlaw.Fit(data)
    print(results.power_law.alpha)
    print(results.power_law.xmin)
    R, p = results.distribution_compare('power_law', 'lognormal')
    return results.power_law.alpha, results.power_law.xmin

def plot_pdf(fit_data):
    pdf = powerlaw.pdf(fit_data)
    return pdf
"""

In [33]:
a,xmin = py"fit_data_powerlaw"(fit_data)

Calculating best minimal value for power law fit
C:\Users\gabipana\.julia\conda\3\lib\site-packages\powerlaw.py:699: RuntimeWarning: invalid value encountered in divide
  (CDF_diff**2) /
C:\Users\gabipana\.julia\conda\3\lib\site-packages\powerlaw.py:699: RuntimeWarning: invalid value encountered in divide
  (CDF_diff**2) /


(2.877396783935865, 16.0)

In [34]:
a, xmin

(2.877396783935865, 16.0)

In [35]:
pdf = py"plot_pdf"(fit_data)

([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 10.0, 12.0, 16.0, 20.0, 25.0, 32.0, 40.0, 51.0, 65.0, 82.0], [0.0017371163867979154, 0.4394904458598726, 0.003474232773595831, 0.14128546612623047, 0.0023161551823972205, 0.04226983207874928, 0.024898668210770122, 0.023451071221771858, 0.01404169079328315, 0.012159814707585408, 0.007411696583671106, 0.0038878319133096202, 0.00253329473074696, 0.002158235510870137, 0.0009926379353130945, 0.00020436663374093125])

In [36]:
function power_law(x,b,a)
    return b*x .^(-a)
end

power_law (generic function with 1 method)

In [37]:
pdf[1]

17-element Vector{Float64}:
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  8.0
 10.0
 12.0
 16.0
 20.0
 25.0
 32.0
 40.0
 51.0
 65.0
 82.0

In [38]:
findfirst(item -> item == 16.0, pdf[1])

10

In [39]:
pdf_cutoff = pdf[1][findfirst(item -> item == 16.0, pdf[1]) : end]

8-element Vector{Float64}:
 16.0
 20.0
 25.0
 32.0
 40.0
 51.0
 65.0
 82.0

In [41]:
scatter(pdf,xscale=:log10, yscale=:log10)
plot!(pdf_cutoff,power_law(pdf_cutoff,xmin,a))
savefig("./results/vrancea_fit_mle.png")

"d:\\projects\\seismicNetworks-extras\\seismic-networks-julia-tests\\results\\vrancea_fit_mle.png"